In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import scipy.optimize
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.gridspec import GridSpec
from tqdm import tqdm
import pandas as pd
import random
from IPython.display import clear_output
import time
import dill
try:
    from pyDOE import lhs
except:
    !pip install pyDOE
    from pyDOE import lhs

In [ ]:
from tensorflow.keras.saving import register_keras_serializable

class neural_net(tf.keras.Sequential):
    
    def __init__(self, capas, act, init):
        super(neural_net, self).__init__()

    #--------------------------------------------------------------------------------------------------------------------------------------------------

    # Inicialización de la clase

        self.acts = {"swish": self.swish,
                     "mish": self.mish,
                     "ReLU": self.ReLU}

        self.act = act

        self.init = init

        self.capas = capas

    # Construcción de la red

        self.add(tf.keras.layers.InputLayer(input_shape = (self.capas[0],)))
        
        if self.act == "mish":

            self.mensaje = "Entrada escalada a [-1,1] para usar " + self.act
            
            self.add(tf.keras.layers.Lambda(lambda X: 2.0*(X)/(1)-1.0))
            
        
        if self.act == "swish":

            self.mensaje = "Entrada escalada a [0,1] para usar " + self.act
            
            self.add(tf.keras.layers.Lambda(lambda X: X))
        

        for ancho in self.capas[1:-1]:

            self.add(tf.keras.layers.Dense(ancho, activation = self.acts[self.act], kernel_initializer = self.init))

        self.add(tf.keras.layers.Dense(self.capas[-1], activation = None, kernel_initializer = self.init ))


    # Guardar en listas los tamaños de los pesos w y los sesgos b

        self.sizes_w = []
        self.sizes_b = []

        for i in range(len(self.capas)-1):

            self.sizes_w.append(self.capas[i]*self.capas[i+1])
            self.sizes_b.append(self.capas[i+1])


    #---------------------------------------------------------------------------------------------------------------------------------------------------------

    # Definición de funciones de activación
    
    def swish(self,x):
        return x*tf.math.sigmoid(x)


    def mish(self,x):
        return x*tf.nn.tanh(tf.math.softplus(x))


    def ReLU(self,x):
        return tf.nn.relu(x)

    #--------------------------------------------------------------------------------------------------------------------------------------------------------
    # Guardar los pesos en y sesgos en una unica lista ordenada donde primero van los pesos y luego los sesgos

    def obtener_pesos(self):

        w = []

        for i in range(1,len(self.capas)):

            pesos = []
            sesgos = []

            pesos.extend(self.layers[i].get_weights()[0].flatten())
            sesgos.extend(self.layers[i].get_weights()[1])
            w.extend(pesos + sesgos)

        w = np.copy(w, order = "F")

        return w

    #-------------------------------------------------------------------------------------------------------------------------------------------------------------

    # Establecer pesos capa por capa

    def establecer_pesos(self, w):

        suma_acum = np.cumsum(np.array(self.sizes_b[0:]) + np.array(self.sizes_w[0:]))
        suma_acum = np.insert(suma_acum, 0 , 0)

        for i in range(1, len(self.capas)):

            pesos_sesgos = self.obtener_pesos()[suma_acum[i-1]:suma_acum[i]]
            pesos = pesos_sesgos[0:self.sizes_w[i-1]]
            sesgos = pesos_sesgos[self.sizes_w[i-1]:self.sizes_w[i-1] + self.sizes_b[i-1]]

            pesos = tf.reshape(pesos, [self.capas[i-1], self.capas[i]])

            pesos_sesgos = [pesos, sesgos]

            self.layers[i].set_weights(pesos_sesgos)



    def set_weights_2(self, w):
        for i, layer in enumerate(self.layers[1:]):
            start_weights = sum(self.sizes_w[:i]) + sum(self.sizes_b[:i])
            end_weights = sum(self.sizes_w[:i+1]) + sum(self.sizes_b[:i])
            weights = w[start_weights:end_weights]
            w_div = int(self.sizes_w[i] / self.sizes_b[i])
            weights = tf.reshape(weights, [w_div, self.sizes_b[i]])
            biases = w[end_weights:end_weights + self.sizes_b[i]]
            weights_biases = [weights, biases]
            layer.set_weights(weights_biases)

In [ ]:
#Clase para derivar las salidas de la red neuronal

class gradientes(tf.keras.layers.Layer):

    def __init__(self, modelo):

        self.modelo = modelo

    def der(self, xy):

        xy = tf.convert_to_tensor(xy)

        x, y = [xy[:, i, tf.newaxis] for i in range(xy.shape[-1])]

        with tf.GradientTape(persistent = True) as dd:
            
            dd.watch(x)
            dd.watch(y)

            with tf.GradientTape(persistent = True) as d:

                d.watch(x)
                d.watch(y)

                u_v_p = self.modelo(tf.concat([x,y], axis=-1))

                u = tf.cast(u_v_p[:, 0, tf.newaxis], dtype=tf.float32)
                v = tf.cast(u_v_p[:, 1, tf.newaxis], dtype=tf.float32)
                p = tf.cast(u_v_p[:, 2, tf.newaxis], dtype=tf.float32)


            u_x = d.gradient(u, x)
            u_x = tf.cast(u_x, dtype=tf.float32)

            v_x = d.gradient(v, x)
            v_x = tf.cast(v_x, dtype=tf.float32)  

            p_x = d.gradient(p, x)
            p_x = tf.cast(p_x, dtype=tf.float32)

            u_y = d.gradient(u, y)
            u_y = tf.cast(u_y, dtype=tf.float32)

            v_y = d.gradient(v, y)
            v_y = tf.cast(v_y, dtype=tf.float32)  

            p_y = d.gradient(p, y)
            p_y = tf.cast(p_y, dtype=tf.float32)  

            del d

        u_xx = dd.gradient(u_x, x)
        u_xx = tf.cast(u_xx, dtype=tf.float32)
        
        u_yy = dd.gradient(u_y, y)
        u_yy = tf.cast(u_yy, dtype=tf.float32)

        v_xx = dd.gradient(v_x, x)
        v_xx = tf.cast(v_xx, dtype=tf.float32)
        
        v_yy = dd.gradient(v_y, y)
        v_yy = tf.cast(v_yy, dtype=tf.float32)

        del dd

        u_grads = u, u_x, u_y, u_xx, u_yy
        v_grads = v, v_x, v_y, v_xx, v_yy
        p_grads = p, p_x, p_y

        return u_grads, v_grads, p_grads

In [ ]:
class training:
    
    def __init__(self, modelo, xyt, xyu, xyd, xyl, xyr):


        self.modelo = modelo
        self.xyt = xyt
        self.xyu = xyu
        self.xyd = xyd
        self.xyl = xyl
        self.xyr = xyr
        self.training_loss = []
        self.valida_loss = []
        self.save_iter = []
        self.Atest = []
        self.Btest = []
        self.Ctest = []
        self.Dtest = []
        self.Etest = []
        self.save_iter_2 = []
        self.final_time = []
        self.inicial_iter = 0
        self.save_time = []
#--------------------------------------------------------------------------------------------------------------------------------------------------------


    def loss(self):
        loss = self.modelo.per(self.xyt, self.xyu, self.xyd, self.xyl, self.xyr)
        return loss


    def perdida_y_grads(self, w):

        with tf.GradientTape() as d: 

            self.modelo.set_weights_2(w)

            loss_value = self.loss()

        grads = d.gradient(loss_value, self.modelo.trainable_variables)

        grad_flat = []

        for g in grads:

            grad_flat.append(tf.reshape(g, [-1]))

        grad_flat = tf.concat(grad_flat, 0)

        loss_value = np.copy(loss_value, order='F')
        grad_flat = np.copy(grad_flat, order='F')

        return loss_value, grad_flat


    #--------------------------------------------------------------------------------------------------------------------------------------------------
    @tf.function
    def adam_train_step(self, optimizer = tf.keras.optimizers.Adam()):

        with tf.GradientTape() as tape:

            perdida = self.loss()

        grads = tape.gradient(perdida, self.modelo.trainable_variables)

        optimizer.apply_gradients(zip(grads, self.modelo.trainable_variables))

        return perdida
    

    def train(self, iteraciones_adam, iteraciones_LBFGS_max, k=1):
        
        
        adam_hist = []

        if (iteraciones_adam):

            print("~~ Optimización Adam ~~")

            start_time = time.time()

            iteration_start_time = start_time

            for i in range(iteraciones_adam):

                current_loss = self.adam_train_step()

                adam_hist.append(current_loss)

                iteration_time = str(time.time() - iteration_start_time)[:5]
            
                print(f'\rLoss: {current_loss} ; time: {iteration_time} ; iter: {i+1} / {iteraciones_adam}', end='')

                iteration_start_time = time.time()

            tiempo_total = (time.time() - start_time)

            print()
            print()

            print('Training time: %.4f segundos' % (tiempo_total))
            
            self.final_time.append(tiempo_total/60)
            self.save_time.append(str(tiempo_total)[:5])

    #---------------------------------------------------------------------------------------------------------------------------------------------------------


        self.LBFGS_hist = [self.loss()]

        if (iteraciones_LBFGS_max):
            
            
            
            self.xy_interior1 = self.test(101)[0]
            self.xy_bnd_arriba1 = self.test(101)[1]
            self.xy_bnd_abajo1 = self.test(101)[2]
            self.xy_bnd_izquierda1 = self.test(101)[3]
            self.xy_bnd_derecha1 = self.test(101)[4]
            
            self.xy_interior2 = self.test(151)[0]
            self.xy_bnd_arriba2 = self.test(151)[1]
            self.xy_bnd_abajo2 = self.test(151)[2]
            self.xy_bnd_izquierda2 = self.test(151)[3]
            self.xy_bnd_derecha2 = self.test(151)[4]
            
            self.xy_interior3 = self.test(186)[0]
            self.xy_bnd_arriba3 = self.test(186)[1]
            self.xy_bnd_abajo3 = self.test(186)[2]
            self.xy_bnd_izquierda3 = self.test(186)[3]
            self.xy_bnd_derecha3 = self.test(186)[4]
            
            self.xy_interior4 = self.test(221)[0]
            self.xy_bnd_arriba4 = self.test(221)[1]
            self.xy_bnd_abajo4 = self.test(221)[2]
            self.xy_bnd_izquierda4 = self.test(221)[3]
            self.xy_bnd_derecha4 = self.test(221)[4]
            
            self.xy_interior5 = self.test(318)[0]
            self.xy_bnd_arriba5 = self.test(318)[1]
            self.xy_bnd_abajo5 = self.test(318)[2]
            self.xy_bnd_izquierda5 = self.test(318)[3]
            self.xy_bnd_derecha5 = self.test(318)[4]
            
            
            self.iteraciones_LBFGS_max = iteraciones_LBFGS_max

            maxiter = iteraciones_LBFGS_max - self.inicial_iter
            
            self.maxiter = maxiter

            print('~~ Optimización L-BFGS ~~')

            self.iter = self.inicial_iter
            
            if k ==  1:
            
                self.save_iter.append(self.iter)

                self.save_iter_2.append(self.iter)


                self.Atest.append(self.modelo.per(self.xy_interior1, self.xy_bnd_arriba1,
                                                 self.xy_bnd_abajo1, self.xy_bnd_izquierda1,
                                                 self.xy_bnd_derecha1).numpy())

                self.Btest.append(self.modelo.per(self.xy_interior2, self.xy_bnd_arriba2,
                                                 self.xy_bnd_abajo2, self.xy_bnd_izquierda2,
                                                 self.xy_bnd_derecha2).numpy())

                self.Ctest.append(self.modelo.per(self.xy_interior3, self.xy_bnd_arriba3,
                                                 self.xy_bnd_abajo3, self.xy_bnd_izquierda3,
                                                 self.xy_bnd_derecha3).numpy())

                self.Dtest.append(self.modelo.per(self.xy_interior4, self.xy_bnd_arriba4,
                                                 self.xy_bnd_abajo4, self.xy_bnd_izquierda4,
                                                 self.xy_bnd_derecha4).numpy())

                self.Etest.append(self.modelo.per(self.xy_interior5, self.xy_bnd_arriba5,
                                                 self.xy_bnd_abajo5, self.xy_bnd_izquierda5,
                                                 self.xy_bnd_derecha5).numpy())

                self.training_loss.append(self.loss().numpy())  

            inicial_tiempo = time.time()  

            self.t_last_callback = time.time()

            conjunto = [i*100 for i in range(1, 200 + 1)]
            
            conjunto_2 = [i*20 for i in range(1,1000 + 1)]

            self.conjunto = conjunto  
            
            self.conjunto_2 = conjunto_2
            
                
#---------------------------------------------------------------------------------------------------------------------                

            results = scipy.optimize.minimize(self.perdida_y_grads,
                            self.modelo.obtener_pesos(),
                            method='L-BFGS-B',
                            jac = True,
                            callback = self.callback,
                            options = {"maxiter" : self.maxiter,
                                       "maxfun" : 100000,
                                       "maxcor" : 50,
                                       "maxls" : 50,
                                       "ftol" : np.finfo(float).eps})
            
            w_optimal = results.x

            self.modelo.set_weights_2(w_optimal)
            
            tiempo_total_2 = (time.time() - inicial_tiempo)
            
            self.final_time.append(tiempo_total_2/60)
            
            self.final_time.append(self.final_time[0] + self.final_time[1])

            print()
            print()
            print()
            print()

            print("Modelo entrenado", "\n", "Final loss = ", self.training_loss[-1], "; tiempo: " , str(tiempo_total_2)[:6], "segundos")


#-----------------------------------------------------------------------------------------------------------------------
#Función para generar mallas estructuradas uniformes

    def test(self, points):

        x = np.linspace(0, 1, points)
        y = x.copy()

        X, Y = np.meshgrid(x,y)
        xy = np.stack([X[1:-1, 1:-1].flatten(),Y[1:-1, 1:-1].flatten()],axis = -1)

        ceros = np.zeros([points,2])
        unos = np.ones([points,2])

        xy_bnd_arriba = unos.copy()
        xy_bnd_arriba[:,0] = x

        xy_bnd_abajo = ceros.copy()
        xy_bnd_abajo[:,0] = x

        xy_bnd_izquierda = ceros.copy()
        xy_bnd_izquierda[:,1] = x
        xy_bnd_izquierda = xy_bnd_izquierda[1:-1]

        xy_bnd_derecha = unos
        xy_bnd_derecha[:,1] = x
        xy_bnd_derecha = xy_bnd_derecha[1:-1]

        return xy, xy_bnd_arriba, xy_bnd_abajo, xy_bnd_izquierda, xy_bnd_derecha
#------------------------------------------------------------------------------------------------------------------        
        
    def borrar(self,eliminar):

        file_to_delete = eliminar 

        working_dir = '/kaggle/working/'

        file_path = os.path.join(working_dir, file_to_delete)

        if os.path.exists(file_path):
            os.remove(file_path)
            
    
    def escribir_txt(self):
           
        mean_list = list((np.array(self.Atest) + np.array(self.Btest) + np.array(self.Ctest) + np.array(self.Dtest) + np.array(self.Etest))/5).copy()

        indice_optimo = mean_list.index(min(mean_list))

        special_case = indice_optimo*20



        contador_tiempo = (np.sum(np.array(self.save_time).astype(np.float32)[0:special_case])*2-2)/60


        ARCHIVO = "/kaggle/working/" + str(self.iter) + "_visualizador_documento.txt"

        ARCHIVO_ELIMINAR = "/kaggle/working/" + str(self.iter - 20) + "_visualizador_documento.txt"

        self.borrar(ARCHIVO_ELIMINAR)

        with open(ARCHIVO , "w") as archivo: 

            archivo.write("Iteración actual = " + str(self.iter) + "\n")
            archivo.write("Modelo = " + str(self.modelo.capas) + "\n")
            archivo.write("Semilla = " + str(seed) + "\n")
            archivo.write("MSE de entrenamiento = " + str(self.training_loss[special_case]) + "\n")
            archivo.write("MSE del conjunto de prueba A = " + str(self.Atest[indice_optimo]) + "\n")
            archivo.write("MSE del conjunto de prueba B = " + str(self.Btest[indice_optimo]) + "\n")
            archivo.write("MSE del conjunto de prueba C = " + str(self.Ctest[indice_optimo]) + "\n")
            archivo.write("MSE del conjunto de prueba D = " + str(self.Dtest[indice_optimo]) + "\n")
            archivo.write("MSE del conjunto de prueba E = " + str(self.Etest[indice_optimo]) + "\n")
            archivo.write("MSE promedio de los conjuntos de prueba = " + str(min(mean_list)) + "\n")
            archivo.write("Número de iteraciones = " + str(special_case) + "\n")
            archivo.write("Tiempo = " + str(contador_tiempo) + "\n")
            archivo.write("Better training MSE = " + str(min(self.training_loss)) + "\n")
            archivo.write("Iter better training MSE = " + str(self.training_loss.index(min(self.training_loss))) + "\n")


    def callback(self, pars):

        t_iteration = str(time.time() - self.t_last_callback)[:5]
        
        self.save_time.append(t_iteration)

        self.iter = self.iter + 1

        loss_value = self.loss().numpy()

        print(f"\rloss: {loss_value} ; iteracion:  {self.iter} / {self.iteraciones_LBFGS_max} ; tiempo: {t_iteration} segundos", end = "")

        self.training_loss.append(loss_value)

        self.t_last_callback = time.time()
        
        self.save_iter.append(self.iter)
        

#----------------------------------------------------------------------------------------------------------------------------
        
        if self.iter in self.conjunto_2:
            
            self.save_iter_2.append(self.iter)
            
            self.Atest.append(self.modelo.per(self.xy_interior1, self.xy_bnd_arriba1,
                                             self.xy_bnd_abajo1, self.xy_bnd_izquierda1,
                                             self.xy_bnd_derecha1).numpy())

            self.Btest.append(self.modelo.per(self.xy_interior2, self.xy_bnd_arriba2,
                                             self.xy_bnd_abajo2, self.xy_bnd_izquierda2,
                                             self.xy_bnd_derecha2).numpy())

            self.Ctest.append(self.modelo.per(self.xy_interior3, self.xy_bnd_arriba3,
                                             self.xy_bnd_abajo3, self.xy_bnd_izquierda3,
                                             self.xy_bnd_derecha3).numpy())

            self.Dtest.append(self.modelo.per(self.xy_interior4, self.xy_bnd_arriba4,
                                             self.xy_bnd_abajo4, self.xy_bnd_izquierda4,
                                             self.xy_bnd_derecha4).numpy())
            
            self.Etest.append(self.modelo.per(self.xy_interior5, self.xy_bnd_arriba5,
                                             self.xy_bnd_abajo5, self.xy_bnd_izquierda5,
                                             self.xy_bnd_derecha5).numpy())
            
            self.escribir_txt()
            
            
#-----------------------------------------------------------------------------------------------------------------------------        
        
        if self.iter in self.conjunto:
            
            clear_output()
            
            modelo_guardado = '/kaggle/working/'  + "pesos_guardados"

            self.modelo.save_weights( modelo_guardado + ".weights.h5")

            print(f" ------> Punto de guardado: {int(self.iter/100)} / {int(self.iteraciones_LBFGS_max/100)} ;  loss guardada = {loss_value}", end = "")
            
            with open('/kaggle/working/training_loss.pkl', 'wb') as f1:
                dill.dump(self.training_loss, f1)
                
            with open('/kaggle/working/Atest_loss.pkl', 'wb') as f3:
                dill.dump(self.Atest, f3)
            
            with open('/kaggle/working/Btest_loss.pkl', 'wb') as f4:
                dill.dump(self.Btest, f4)
                
            with open('/kaggle/working/Ctest_loss.pkl', 'wb') as f5:
                dill.dump(self.Ctest, f5)
            
            with open('/kaggle/working/Dtest_loss.pkl', 'wb') as f6:
                dill.dump(self.Dtest, f6)
                
            with open('/kaggle/working/Etest_loss.pkl', 'wb') as f7:
                dill.dump(self.Etest, f7)
                
            with open('/kaggle/working/save_iter.pkl', 'wb') as f8:
                dill.dump(self.save_iter, f8)
                
            with open('/kaggle/working/save_iter2.pkl', 'wb') as f9:
                dill.dump(self.save_iter_2, f9)
                
            with open('/kaggle/working/save_time.pkl', 'wb') as f10:
                dill.dump(self.save_time, f10)
                
            with open('/kaggle/working/inicial_iter.pkl', 'wb') as f11:
                dill.dump(self.inicial_iter, f11)
            
            
#------------------------------------------------------------------------------------------------------------------------            
        
            # Actualizar los datos de las líneas de la gráfica
            
            plt.ion()
            self.fig, self.ax = plt.subplots()
            self.training_line, = self.ax.plot([], [], label="MSE de entrenamiento")
            self.Aline, = self.ax.plot([], [], label="MSE del conjunto de prueba A", linestyle = "--")
            self.Bline, = self.ax.plot([], [], label="MSE del conjunto de prueba B", linestyle = "dashdot")
            self.Cline, = self.ax.plot([], [], label="MSE del conjunto de prueba C", linestyle = "dotted")
            self.Dline, = self.ax.plot([], [], label="MSE del conjunto de prueba D")
            self.Eline, = self.ax.plot([], [], label="MSE del conjunto de prueba E", color = "yellow")

            self.fig.suptitle("MSE de entrenamiento y de los conjuntos de prueba en el modelo\n" + str(self.modelo.capas)
                              + " con función de activación " + str(self.modelo.act), x=0.5, ha='center')
            self.ax.legend()
            plt.xlabel("Número de iteraciones L-BFGS-B")
            plt.ylabel("Error cuadrático medio")
            plt.gca().annotate('Total iteraciones = ' + str(self.iter), 
                               xy=(0.6, 0.55), xycoords='axes fraction', fontsize=9, color='black')
            plt.tight_layout()        
        
            
            self.training_line.set_data(self.save_iter, self.training_loss)
            
            self.Aline.set_data(self.save_iter_2, self.Atest)
            self.Bline.set_data(self.save_iter_2, self.Btest)
            self.Cline.set_data(self.save_iter_2, self.Ctest)
            self.Dline.set_data(self.save_iter_2, self.Dtest)
            self.Eline.set_data(self.save_iter_2, self.Etest)
            
            
            # Actualizar los límites de los ejes
            self.ax.relim()
            self.ax.autoscale_view()

            
            self.fig.canvas.draw()
            plt.pause(0.01)  # Esto es necesario para que la gráfica se actualice en tiempo real
            
            self.modelo.capas
            
            global nb, nf
            
            nombre_archivo = '/kaggle/working/'  +  str(self.modelo.capas)[1:-1] + "_" + str(self.modelo.act) + "_" + str(self.modelo.init) + "_" + str(nb) + "_" + str(nf)
        
            self.fig.savefig(nombre_archivo + ".png")

In [ ]:
class navier_stokes(neural_net):
    
#rho es la densidad y mu la viscosidad dinámica

    def __init__(self, net, rho=1, mu=0.01):
        super(navier_stokes, self).__init__(net.capas, net.act, net.init)


        self.net = net
        self.rho = rho
        self.mu = mu
        
        if red.act == "mish":
            print(red.mensaje)
            
        if red.act == "swish":
            print(red.mensaje)


    def deri(self, xy_points):
        return gradientes(pinn).der(xy_points)


    def per(self, xy_eqn, xy_up_bnd, xy_down_bnd, xy_left_bnd, xy_right_bnd):

# EDPs de navier stokes --------------------------------------------------------------------------------------------

        u_grads, v_grads, p_grads = self.deri(xy_eqn)

        u, u_x, u_y, u_xx, u_yy = u_grads

        v, v_x, v_y, v_xx, v_yy = v_grads

        p, p_x, p_y = p_grads



        r_u = u*u_x + v*u_y + p_x/self.rho - self.mu*(u_xx + u_yy) 
        r_v = u*v_x + v*v_y + p_y/self.rho - self.mu*(v_xx + v_yy)
        r_div = u_x + v_y


# Condiciones de contorno ------------------------------------------------------------------------------------------

        u_up, v_up = pinn(xy_up_bnd)[:,0, tf.newaxis], pinn(xy_up_bnd)[:,1, tf.newaxis]

        u_down, v_down = pinn(xy_down_bnd)[:,0, tf.newaxis], pinn(xy_down_bnd)[:,1, tf.newaxis]

        u_left, v_left = pinn(xy_left_bnd)[:,0, tf.newaxis], pinn(xy_left_bnd)[:,1, tf.newaxis]

        u_right, v_right = pinn(xy_right_bnd)[:,0, tf.newaxis], pinn(xy_right_bnd)[:,1, tf.newaxis]
    
# Residual de las condiciones de contorno --------------------------------------------------------------------------------------------

        yB = tf.reduce_mean(tf.square(u_up - 1.0)) + tf.reduce_mean(tf.square(v_up)) + \
        tf.reduce_mean(tf.square(u_down)) + tf.reduce_mean(tf.square(v_down)) + \
        tf.reduce_mean(tf.square(u_left)) + tf.reduce_mean(tf.square(v_left)) + \
        tf.reduce_mean(tf.square(u_right)) + tf.reduce_mean(tf.square(v_right))
    
#Residual de las EDPs --------------------------------------------------------------------------

        yS = tf.reduce_mean(tf.square(r_u)) + tf.reduce_mean(tf.square(r_v)) + \
        tf.reduce_mean(tf.square(r_div))

#Función de costo -------------------------------------------------------------------------------------------------
        y = yS + yB
#------------------------------------------------------------------------------------------------------------------
        return y

**GENERAR DATOS DE ENTRENAMIENTO**

In [ ]:
seed = 8
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

nf = 35000 #Puntos en el interior del dominio
nb = 500 #Puntos por pared

#Función para generar los datos de entrenamiento
def gen_training_data(Nf = nf, Nb = nb):

    #Muestreo con hipercubo latino

    xy_train = lhs(2,Nf)
    
    #Arreglos de ceros y unos

    unos = np.ones([Nb,2])
    ceros = np.zeros([Nb,2])

    #Muestreo aleatorio para las condiciones de contorno

    xy_up = unos.copy()
    xy_up[:,0] = np.random.rand(Nb)

    xy_down = ceros.copy()
    xy_down[:,0] = np.random.rand(Nb)

    xy_left = ceros.copy()
    xy_left[:,1] = np.random.rand(Nb)

    xy_right = unos.copy()
    xy_right[:,1] = np.random.rand(Nb)
    
    return xy_train ,xy_up, xy_down, xy_left, xy_right

#Datos de entrenamiento
xy_train = gen_training_data()[0]
xy_up = gen_training_data()[1]
xy_down = gen_training_data()[2]
xy_left = gen_training_data()[3]
xy_right = gen_training_data()[4]

In [ ]:
seed = 11
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

MODELO = [2, 130, 3]

red = neural_net(MODELO, act = "swish",  init = "he_normal")

pinn = navier_stokes(red)

entrenamiento = training(pinn, xy_train ,xy_up, xy_down, xy_left, xy_right)

entrenamiento.train(1500, 20000)